In [ ]:
# from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import ifft, fft, fftfreq, fftshift, ifft2, fft2
from scipy import signal
from scipy.io import wavfile
import plotly
import torch
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
from datetime import datetime, timedelta
from audio_function import plot1, plot1_f, mean_data_sep, mean_autocorr_signal, corr_t, corr_f, count_time, cos_sim, cr_arr_t, norm_max, load_mat, first_second_part, filt_freq, variation_data, t_arr_for_corr_t
from audio_function import df, audio_data49_3, audio_data49_2, audio_data16_3, audio_data16_2, sample_rate, audio_data49_1

In [ ]:
data_0db = load_mat('/Users/bogda/Desktop/Acoustics/Scientific/Исходные данные/Matlab_Files/2023.03.31-11.59.31_channel_0.mat')
data_70db = load_mat('/Users/bogda/Desktop/Acoustics/Scientific/Исходные данные/Matlab_Files/2023.03.31-11.52.03_channel_0.mat')
sample_rate = 48000
data_first, data_second = first_second_part(data_0db)
mn = min(len(data_first), len(data_second))
t = np.arange(int(mn))/sample_rate
t_central = t - mn/sample_rate/2
f = fftfreq(len(data_0db), 1 / sample_rate)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(f, np.abs(ifft(data_0db)), color='blue')
plt.grid(True)
plt.xlim(0, 1200)

plt.show()

In [ ]:
begin = int(10*sample_rate)
end = int(10*sample_rate+(sample_rate*25/100))
data_0db_sec_10 = data_0db[begin:end]

begin = int(sample_rate)
end = int(sample_rate*125/100)
data_0db_sec_1 = data_0db[begin:end]
f_1 = fftfreq(int(end-begin), 1 / sample_rate)
plt.figure(figsize=(10,4))
plt.plot(f_1, np.abs(ifft(data_0db_sec_1)), color='blue')
plt.plot(f_1, np.abs(ifft(data_0db_sec_10)), color='red')
plt.grid(True)
plt.xlim(0, 1200)

plt.show()

In [ ]:
# Параметры для гауссовых функций
centers = np.arange(230, 2000, 230)  
amplitudes = np.array([100, 100, 100, 100, 100, 100 , 100, 100, 100, 100, 100, 100])  
width = 4


def gaussian(x, center, amplitude, width):
    return amplitude * np.exp(-0.5 * ((x - center) / width) ** 2)

spectrum_gaus = np.zeros_like(f_1) + 0.0001

# Добавляем Гауссовы пики
for center, amplitude in zip(centers, amplitudes):
    spectrum_gaus += gaussian(f_1, center, amplitude, width)
# spectrum_gaus_db = 20*np.log10(spectrum_gaus*1000)

# a = []
# for i in range (0, len(data_0db)//sample_rate):
#     i_beg = int(i*sample_rate)
#     i_fin = int((i+1)*sample_rate)
#     data_0db_sec = data_0db[i_beg:i_fin]
#     data_0db_db = 20*np.log10(np.abs(ifft(data_0db_sec))*1000)
#     grad_data_0db_db = np.gradient(data_0db_db, f_1[0])
#     grad_gaus_spec = np.gradient(spectrum_gaus_db, f_1)
#     summarize = np.sum(grad_data_0db_db*grad_gaus_spec)
#     a.append(summarize)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(f_1, spectrum_gaus, color='blue')
plt.grid(True)
plt.xlim(0, 1000)
plt.show()

In [ ]:
grad_gaus_spec = np.gradient(spectrum_gaus, f_1)
plt.figure(figsize=(10,4))
plt.plot(f_1, grad_gaus_spec, color='blue')
plt.grid(True)
plt.xlim(0, 1000)
plt.show()

In [ ]:
begin = int(sample_rate)
end = int(sample_rate*125/100)
data_0db_sec = data_0db[begin:end] 
spectrum = np.abs(ifft(data_0db_sec))
# f_1 = fftfreq(end-begin, 1 / sample_rate)
grad_spec = []
for i in range(0, len(spectrum)-1):
    c = (spectrum[i+1] - spectrum[i]) / (f_1[i+1] - f_1[i])
    grad_spec.append(c)
f_1 = f_1[:len(grad_spec)]

plt.figure(figsize=(10,4))
plt.plot(f_1, grad_spec, color='blue')
plt.grid(True)
plt.xlim(0, 1000)
plt.show()

In [ ]:
grad_gaus_spec = grad_gaus_spec[:len(grad_spec)]
A = np.sum(grad_spec*grad_gaus_spec)
print(A)

In [ ]:
seconds = np.arange(len(data_0db)//sample_rate)
plt.figure(figsize=(10,4))
plt.plot(seconds, a, color='blue')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(f, np.abs(ifft(data_0db)), color='blue')
plt.grid(True)
plt.xlim(220, 240)

plt.show()

In [ ]:
import numpy as np
import scipy.signal as signal

def generate_noise(frequencies, noise_type="white", seed=None):
    np.random.seed(seed)
    num_samples = len(frequencies)
    
    # Генерация белого шума (равномерный спектр)
    white_noise = 30*np.random.randn(num_samples)
    
    if noise_type == "white":
        return white_noise
    
    # Генерация розового шума (1/f спектр)
    elif noise_type == "pink":
        
        pink_noise = white/f_1
        return pink_noise
    
    # Генерация красного (brownian) шума (1/f^2 спектр)
    elif noise_type == "red":
        uneven = num_samples % 2
        X = np.random.randn(num_samples // 2 + 1 + uneven) + 1j * np.random.randn(num_samples // 2 + 1 + uneven)
        S = np.fft.rfftfreq(num_samples, d=1.0)
        S[0] = 1  # защита от деления на 0
        X /= S  # Применяем фильтр 1/f^2
        red_noise = np.fft.irfft(X)[:num_samples]
        return red_noise
    
    else:
        raise ValueError("Unknown noise type. Choose from 'white', 'pink', or 'red'.")

# Пример использования
frequencies = f_1
white = generate_noise(frequencies, "white")
pink = generate_noise(frequencies, "pink")
red = generate_noise(frequencies, "red")

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(f_1, white, color='blue')
plt.grid(True)
plt.xlim(1000, 1200)

plt.show()

In [ ]:
b = []
for i in range (0, len(data_0db)//sample_rate):
    i_beg = int(i*sample_rate)
    i_fin = int((i+1)*sample_rate)
    data_0db_sec = data_0db[i_beg:i_fin]
    data_0db_db = 20*np.log10(np.abs(ifft(data_0db_sec)+white)*1000)
    grad_data_0db_db = np.gradient(data_0db_db, f_1)
    grad_gaus_spec = np.gradient(spectrum_gaus_db, f_1)
    summarize = np.sum(grad_data_0db_db*grad_gaus_spec)
    b.append(summarize)

In [ ]:
seconds = np.arange(len(data_0db)//sample_rate)
plt.figure(figsize=(10,4))
plt.plot(seconds, b, color='blue')
plt.grid(True)
plt.show()